In [3]:
import pandas as pd
import math
import collections
import datetime


# 국가, 날짜별 치사율 데이터 가져오기

#### 데이터 소스: https://github.com/owid/covid-19-data/tree/master/public/data

In [4]:
data = pd.read_csv("./data/owid-covid-data.csv")
data

,iso_code,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,total_deaths_per_million,...,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cvd_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_100k
0,ABW,Aruba,2020-03-13,2,2,0,0,18.733,18.733,0.0,...,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN
1,ABW,Aruba,2020-03-20,4,2,0,0,37.465,18.733,0.0,...,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN
2,ABW,Aruba,2020-03-24,12,8,0,0,112.395,74.930,0.0,...,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN
3,ABW,Aruba,2020-03-25,17,5,0,0,159.227,46.831,0.0,...,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN
4,ABW,Aruba,2020-03-26,19,2,0,0,177.959,18.733,0.0,...,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17420,NaN,International,2020-02-28,705,0,4,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17421,NaN,International,2020-02-29,705,0,6,2,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17422,NaN,International,2020-03-01,705,0,6,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17423,NaN,International,2020-03-02,705,0,6,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 국가 filtering하기

In [5]:
countrySeries = data.iso_code
countryList = list(set(countrySeries)) # set으로 만들어서 중복 다 없앰.

len(countryList)

212

In [6]:

caseData = data.total_cases

In [7]:
selectedCountries = []
countryRowList = []
for country in countryList:
    check = False
    for i in range(len(countrySeries)):
        if(country == countrySeries[i]):
            check = True
        if(check and country != countrySeries[i]): # 맨 마지막 total값을 가져오기 위해서 country[i]이름이 달라지면 i-1을 리스트에 넣는다.
            if(caseData[i-1] > 3000 and country != 'OWID_WRL'): # case가 3000명이 넘는 것만 들고 옴, OWID_WRL은 전세계 데이터라 뺌
                countryRowList.append(i-1)
                selectedCountries.append(country)
            break
            
len(countryRowList) 

68

In [8]:
selectedCountries

['ARE',
 'COL',
 'NOR',
 'EGY',
 'CAN',
 'IRN',
 'ARM',
 'CZE',
 'BEL',
 'FRA',
 'SGP',
 'SWE',
 'SRB',
 'AUS',
 'DNK',
 'ARG',
 'MDA',
 'SAU',
 'MAR',
 'AUT',
 'PAK',
 'MYS',
 'BHR',
 'PER',
 'ITA',
 'BGD',
 'IRQ',
 'NLD',
 'THA',
 'ZAF',
 'KOR',
 'NGA',
 'ECU',
 'ISR',
 'ESP',
 'LUX',
 'FIN',
 'KWT',
 'CHN',
 'DZA',
 'BOL',
 'IND',
 'AFG',
 'PHL',
 'BRA',
 'USA',
 'MEX',
 'KAZ',
 'TUR',
 'BLR',
 'PRT',
 'POL',
 'DEU',
 'UKR',
 'IDN',
 'DOM',
 'GHA',
 'QAT',
 'ROU',
 'GBR',
 'JPN',
 'HUN',
 'OMN',
 'CHL',
 'RUS',
 'PAN',
 'CHE',
 'IRL']

### 일별 치사율 구하기

In [9]:

data = data[["iso_code",'date', 'total_cases', 'total_deaths']] 
fatalityDataList = data.values.tolist()

In [10]:
fatalityData = []
for row in fatalityDataList:
    if(row[0] in selectedCountries):
        if(row[2] != 0):
            row.append(100* (row[3]/row[2]))
        else:
            row.append(0)

        del row[3]
        del row[2]
        fatalityData.append(row)

In [11]:
fatalityData # 맨 마지막 column이 치사율임

[['AFG', '2019-12-31', 0],
 ['AFG', '2020-01-01', 0],
 ['AFG', '2020-01-02', 0],
 ['AFG', '2020-01-03', 0],
 ['AFG', '2020-01-04', 0],
 ['AFG', '2020-01-05', 0],
 ['AFG', '2020-01-06', 0],
 ['AFG', '2020-01-07', 0],
 ['AFG', '2020-01-08', 0],
 ['AFG', '2020-01-09', 0],
 ['AFG', '2020-01-10', 0],
 ['AFG', '2020-01-11', 0],
 ['AFG', '2020-01-12', 0],
 ['AFG', '2020-01-13', 0],
 ['AFG', '2020-01-14', 0],
 ['AFG', '2020-01-15', 0],
 ['AFG', '2020-01-16', 0],
 ['AFG', '2020-01-17', 0],
 ['AFG', '2020-01-18', 0],
 ['AFG', '2020-01-19', 0],
 ['AFG', '2020-01-20', 0],
 ['AFG', '2020-01-21', 0],
 ['AFG', '2020-01-22', 0],
 ['AFG', '2020-01-23', 0],
 ['AFG', '2020-01-24', 0],
 ['AFG', '2020-01-25', 0],
 ['AFG', '2020-01-26', 0],
 ['AFG', '2020-01-27', 0],
 ['AFG', '2020-01-28', 0],
 ['AFG', '2020-01-29', 0],
 ['AFG', '2020-01-30', 0],
 ['AFG', '2020-01-31', 0],
 ['AFG', '2020-02-01', 0],
 ['AFG', '2020-02-02', 0],
 ['AFG', '2020-02-03', 0],
 ['AFG', '2020-02-04', 0],
 ['AFG', '2020-02-05', 0],
 

# Factor 데이터 time-series로 바꾸기

#### 데이터 소스: https://ourworldindata.org/grapher/full-list-total-tests-for-covid-19

In [12]:
data = pd.read_csv("./data/검사량/full-list-total-tests-for-covid-19.csv")
data

,Entity,Code,Date,Total tests
0,Argentina,ARG,2020-04-08,13330
1,Argentina,ARG,2020-04-09,14850
2,Argentina,ARG,2020-04-10,16379
3,Argentina,ARG,2020-04-11,18027
4,Argentina,ARG,2020-04-13,19758
...,...,...,...,...
4704,Zimbabwe,ZWE,2020-05-09,9593
4705,Zimbabwe,ZWE,2020-05-10,9872
4706,Zimbabwe,ZWE,2020-05-11,10598
4707,Zimbabwe,ZWE,2020-05-12,11058


### 누적데이터 일별 데이터로 바꾸기

In [13]:
cumulativeTestNum = data[['Code', 'Date','Total tests']]
cumulativeTestNumList = cumulativeTestNum.values.tolist()

In [14]:
testNumList= []
curCountry= ''
curDateText = ''
curValue = 0
for row in cumulativeTestNumList:
    prevCountry = curCountry 
    curCountry = row[0]
    if(prevCountry == curCountry):
        prevDateText = curDateText
        prevDate = datetime.datetime.strptime(prevDateText, '%Y-%m-%d').date()
    curDateText = row[1]
    curDate = datetime.datetime.strptime(curDateText, '%Y-%m-%d').date()
    prevValue = curValue
    curValue = row[2]
    
    numOfTest = curValue - prevValue
    if(prevCountry == curCountry):
        if(prevDate + datetime.timedelta(days=1) != curDate):
            timeDelta = (curDate - prevDate).days
            avgTest = int(numOfTest/timeDelta)
            for i in range(timeDelta):
                date = prevDate + datetime.timedelta(days=i+1)
                testNumList.append([prevCountry, date.strftime("%Y-%m-%d"), avgTest])
        else:
            testNumList.append([prevCountry, curDateText, numOfTest])

testNumList

[['ARG', '2020-04-09', 1520],
 ['ARG', '2020-04-10', 1529],
 ['ARG', '2020-04-11', 1648],
 ['ARG', '2020-04-12', 865],
 ['ARG', '2020-04-13', 865],
 ['ARG', '2020-04-14', 3047],
 ['ARG', '2020-04-15', 1569],
 ['ARG', '2020-04-16', 2083],
 ['ARG', '2020-04-17', 2193],
 ['ARG', '2020-04-18', 2292],
 ['ARG', '2020-04-19', 1770],
 ['ARG', '2020-04-20', 1856],
 ['ARG', '2020-04-21', 2043],
 ['ARG', '2020-04-22', 2617],
 ['ARG', '2020-04-23', 2558],
 ['ARG', '2020-04-24', 2868],
 ['ARG', '2020-04-25', 2752],
 ['ARG', '2020-04-26', 2499],
 ['ARG', '2020-04-27', 1995],
 ['ARG', '2020-04-28', 1700],
 ['ARG', '2020-04-29', 2458],
 ['ARG', '2020-04-30', 2627],
 ['ARG', '2020-05-01', 2845],
 ['ARG', '2020-05-02', 2336],
 ['ARG', '2020-05-03', 1947],
 ['ARG', '2020-05-04', 2107],
 ['ARG', '2020-05-05', 1798],
 ['ARG', '2020-05-06', 2597],
 ['ARG', '2020-05-07', 2883],
 ['ARG', '2020-05-08', 2703],
 ['ARG', '2020-05-09', 2828],
 ['ARG', '2020-05-10', 2289],
 ['ARG', '2020-05-11', 2140],
 ['ARG', '20

### 나라별로 데이터 묶어서 Correlation구하기

In [18]:
corrDict = {}
testSeries = []
fatalitySeries = []
curCountry= ''

for rowF in fatalityData:
    prevCountry = curCountry
    curCountry = rowF[0]
    if(prevCountry != curCountry and prevCountry != ''):
        corrList = [testSeries, fatalitySeries]
        df = pd.DataFrame(corrList).T
        corr = df.corr(method= 'pearson')

        corrDict[prevCountry] = corr[1][0]
        testSeries = []
        fatalitySeries = []
    
    for rowT in testNumList:
        if(rowT[0] == rowF[0] and rowT[1] == rowF[1]):
            testSeries.append(rowT[2])
            fatalitySeries.append(rowF[2])

In [19]:
print("검사량 Correlation: ")
corrDict

검사량 Correlation: 


{'AFG': nan,
 'ARE': nan,
 'ARG': 0.5313472976043437,
 'ARM': nan,
 'AUS': 0.6484951652375964,
 'AUT': 0.4782389156115664,
 'BEL': 0.8596375967457597,
 'BGD': -0.36127194527653045,
 'BHR': -0.1864671165872882,
 'BLR': 0.36315857032623,
 'BOL': 0.28250116159553756,
 'BRA': nan,
 'CAN': 0.4574769303934466,
 'CHE': 0.5004122249541416,
 'CHL': 0.32878081131375453,
 'CHN': nan,
 'COL': 0.8325649381355059,
 'CZE': 0.829944341803931,
 'DEU': 0.35136312769254263,
 'DNK': 0.7074837006903386,
 'DOM': nan,
 'DZA': nan,
 'ECU': 0.08857493447153773,
 'EGY': nan,
 'ESP': 0.7789846102842126,
 'FIN': 0.7423715724053329,
 'FRA': 0.76906278411412,
 'GBR': 0.7115133270330045,
 'GHA': -0.3996031801505714,
 'HUN': 0.872117976174676,
 'IDN': -0.06931331906385525,
 'IND': 0.7281665253717498,
 'IRL': 0.8438655319840499,
 'IRN': -0.39743341011553734,
 'IRQ': nan,
 'ISR': 0.806172951961817,
 'ITA': 0.9021153399019721,
 'JPN': -0.045806104936584095,
 'KAZ': 0.4174067902038771,
 'KOR': 0.3550906539748253,
 'KWT':

### 평균 Correlation구하기

In [20]:
count = 0
total = 0
for elem in list(corrDict.values()):
    if not math.isnan(elem):
        count +=1
        total += elem
total/count

0.42031753096594504